In [1]:
# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np

In [3]:
n = 5
for i in range(0, 1<<n):
    gray = i^(i>>1)
    print("{:4d}".format(i),"{0:0{1}b}".format(gray,n),"{:4d}".format(gray))

   0 00000    0
   1 00001    1
   2 00011    3
   3 00010    2
   4 00110    6
   5 00111    7
   6 00101    5
   7 00100    4
   8 01100   12
   9 01101   13
  10 01111   15
  11 01110   14
  12 01010   10
  13 01011   11
  14 01001    9
  15 01000    8
  16 11000   24
  17 11001   25
  18 11011   27
  19 11010   26
  20 11110   30
  21 11111   31
  22 11101   29
  23 11100   28
  24 10100   20
  25 10101   21
  26 10111   23
  27 10110   22
  28 10010   18
  29 10011   19
  30 10001   17
  31 10000   16


In [4]:
def count_bits(bit):
    count = bit
## 32 bits
#    count = (count & 0x55555555) + ((count >> 1) & 0x55555555)
#    count = (count & 0x33333333) + ((count >> 2) & 0x33333333)
#    count = (count & 0x0F0F0F0F) + ((count >> 4) & 0x0F0F0F0F)
#    count = (count & 0x00FF00FF) + ((count >> 8) & 0x00FF00FF)
#    count = (count & 0x0000FFFF) + ((count >>16) & 0x0000FFFF)
## 64 bits
    count = (count & 0x5555555555555555) + ((count & 0xAAAAAAAAAAAAAAAA) >> 1)
    count = (count & 0x3333333333333333) + ((count & 0xCCCCCCCCCCCCCCCC) >> 2)
    count = (count & 0x0F0F0F0F0F0F0F0F) + ((count & 0xF0F0F0F0F0F0F0F0) >> 4)
    count = (count & 0x00FF00FF00FF00FF) + ((count & 0xFF00FF00FF00FF00) >> 8)
    count = (count & 0x0000FFFF0000FFFF) + ((count & 0xFFFF0000FFFF0000) >> 16)
    count = (count & 0x00000000FFFFFFFF) + ((count & 0xFFFFFFFF00000000) >> 32)
    return count

In [5]:
def gray_code(i,n):
    gray = i^(i>>1)
    bit_gray = np.array(list(np.binary_repr(gray).zfill(n))+[count_bits(gray)]).astype(np.int64)
    return gray, count_bits(gray), bit_gray

In [6]:
n = 5
for i in range(0, 1<<n):
    gray, nbit, bit_gray = gray_code(i,n)
    print("{:4d}".format(i),"{0:0{1}b}".format(gray,n),"{:4d}".format(gray),nbit,bit_gray)

   0 00000    0 0 [0 0 0 0 0 0]
   1 00001    1 1 [0 0 0 0 1 1]
   2 00011    3 2 [0 0 0 1 1 2]
   3 00010    2 1 [0 0 0 1 0 1]
   4 00110    6 2 [0 0 1 1 0 2]
   5 00111    7 3 [0 0 1 1 1 3]
   6 00101    5 2 [0 0 1 0 1 2]
   7 00100    4 1 [0 0 1 0 0 1]
   8 01100   12 2 [0 1 1 0 0 2]
   9 01101   13 3 [0 1 1 0 1 3]
  10 01111   15 4 [0 1 1 1 1 4]
  11 01110   14 3 [0 1 1 1 0 3]
  12 01010   10 2 [0 1 0 1 0 2]
  13 01011   11 3 [0 1 0 1 1 3]
  14 01001    9 2 [0 1 0 0 1 2]
  15 01000    8 1 [0 1 0 0 0 1]
  16 11000   24 2 [1 1 0 0 0 2]
  17 11001   25 3 [1 1 0 0 1 3]
  18 11011   27 4 [1 1 0 1 1 4]
  19 11010   26 3 [1 1 0 1 0 3]
  20 11110   30 4 [1 1 1 1 0 4]
  21 11111   31 5 [1 1 1 1 1 5]
  22 11101   29 4 [1 1 1 0 1 4]
  23 11100   28 3 [1 1 1 0 0 3]
  24 10100   20 2 [1 0 1 0 0 2]
  25 10101   21 3 [1 0 1 0 1 3]
  26 10111   23 4 [1 0 1 1 1 4]
  27 10110   22 3 [1 0 1 1 0 3]
  28 10010   18 2 [1 0 0 1 0 2]
  29 10011   19 3 [1 0 0 1 1 3]
  30 10001   17 2 [1 0 0 0 1 2]
  31 100

In [7]:
def slow_gray_code(n,dim):
    res = np.zeros(dim+1,dtype=np.int64)
    pos = dim - 1
    while n>0:
        res[pos] = n%2
        res[dim] += res[pos]
        n = n//2
        pos = pos-1
    return res

In [8]:
n = 5
for i in range(0, 1<<n):
    gray = slow_gray_code(i,n)
    print(gray)

[0 0 0 0 0 0]
[0 0 0 0 1 1]
[0 0 0 1 0 1]
[0 0 0 1 1 2]
[0 0 1 0 0 1]
[0 0 1 0 1 2]
[0 0 1 1 0 2]
[0 0 1 1 1 3]
[0 1 0 0 0 1]
[0 1 0 0 1 2]
[0 1 0 1 0 2]
[0 1 0 1 1 3]
[0 1 1 0 0 2]
[0 1 1 0 1 3]
[0 1 1 1 0 3]
[0 1 1 1 1 4]
[1 0 0 0 0 1]
[1 0 0 0 1 2]
[1 0 0 1 0 2]
[1 0 0 1 1 3]
[1 0 1 0 0 2]
[1 0 1 0 1 3]
[1 0 1 1 0 3]
[1 0 1 1 1 4]
[1 1 0 0 0 2]
[1 1 0 0 1 3]
[1 1 0 1 0 3]
[1 1 0 1 1 4]
[1 1 1 0 0 3]
[1 1 1 0 1 4]
[1 1 1 1 0 4]
[1 1 1 1 1 5]
